# Are abstractions used?

In [39]:
import sys

sys.path.append("../")

%load_ext autoreload
%autoreload 2

import os

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# import data.drawings.make_tasks as drawing_tasks
from src.analysis_utilities import SynthesisExperimentAnalyzer
from src.config_builder import ExperimentType


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
EXPERIMENT_NAME = "runs_gpt_learner"
DOMAIN = "re2"
COMPUTE_LIKELIHOODS = True
EXPERIMENT_TYPE = ["gpt_solver_learner_gpt4"]

In [41]:
analyzer = SynthesisExperimentAnalyzer(
    experiment_name=EXPERIMENT_NAME,
    experiment_types = EXPERIMENT_TYPE,
    allow_incomplete_results=True,
    compute_likelihoods=COMPUTE_LIKELIHOODS,
    batch_size = 96,
    domains = [DOMAIN],
    seeds = [111, 222, 333],
    
)

Experiment directory: ../experiments_iterative/outputs/runs_gpt_learner
Available domains: ['re2']


In [42]:
df_results = analyzer.get_synthesis_results()
mask = df_results["programs"].apply(lambda x: x != [])
df_results = df_results[mask]
df_results

,request,task,programs,best_search_time,split,solved,seed,iteration,experiment_type,domain
13,"{'constructor': '->', 'arguments': [{'construc...",re2_train_14_if_there_is_t_replace_that_with_a_x,[{'program': '(lambda (_rflatten (map (lambda ...,NaN,train,True,111,0,gpt_solver_learner_gpt4,re2
20,"{'constructor': '->', 'arguments': [{'construc...",re2_train_21_if_the_word_starts_with_any_lette...,[{'program': '(lambda (_rflatten (cons _n (con...,1.451924,train,True,111,0,gpt_solver_learner_gpt4,re2
31,"{'constructor': '->', 'arguments': [{'construc...",re2_train_32_if_there_is_p_replace_that_with_f_w,[{'program': '(lambda (_rflatten (map (lambda ...,NaN,train,True,111,0,gpt_solver_learner_gpt4,re2
33,"{'constructor': '->', 'arguments': [{'construc...",re2_train_34_if_the_word_ends_with_any_letter_...,[{'program': '(lambda (_rflatten (_rappend _d ...,0.000519,train,True,111,0,gpt_solver_learner_gpt4,re2
40,"{'constructor': '->', 'arguments': [{'construc...",re2_train_41_if_there_is_any_letter_add_v_afte...,[{'program': '(lambda (_rflatten (map (lambda ...,0.000772,train,True,111,0,gpt_solver_learner_gpt4,re2
...,...,...,...,...,...,...,...,...,...,...
1947,"{'constructor': '->', 'arguments': [{'construc...",re2_train_474_if_there_is_t_replace_that_with_o,[{'program': '(lambda (_rflatten (map (lambda ...,NaN,train,True,111,2,gpt_solver_learner_gpt4,re2
1950,"{'constructor': '->', 'arguments': [{'construc...",re2_train_477_if_there_is_s_replace_that_with_i_r,[{'program': '(lambda (_rflatten (map (lambda ...,NaN,train,True,111,2,gpt_solver_learner_gpt4,re2
1957,"{'constructor': '->', 'arguments': [{'construc...",re2_train_484_if_the_word_ends_with_consonant_...,[{'program': '(lambda (if (_rmatch _e (_rtail ...,NaN,train,True,111,2,gpt_solver_learner_gpt4,re2
1959,"{'constructor': '->', 'arguments': [{'construc...",re2_train_486_if_the_word_starts_with_any_lett...,[{'program': '(lambda (_rflatten (cons _b (_rs...,NaN,train,True,111,2,gpt_solver_learner_gpt4,re2


In [43]:
programs = df_results['programs'].tolist()
programs_list = []
for task in programs:
    for program in task:
        programs_list.append(program["program"])

In [44]:
abstraction = analyzer.get_gpt_abstractions_for_domain(domain = DOMAIN, experiment_types = EXPERIMENT_TYPE)
for program in programs_list:
    for name in abstraction['function_name']:
        if name in program:   
            print('🍊')

In [59]:
import json
path = "/home/ubuntu/lilo/experiments_iterative/outputs/runs_gpt_learner/domains/re2/gpt_solver_learner_gpt4/seed_111/gpt_solver_learner_gpt4_96/1/train/gpt_solver_results.json"
with open(path, 'r') as f:
    data = json.load(f)

In [52]:
programs = []
for x in data["results_by_query"]:
    answer = x["completion"]["choices"]
    for y in answer:
        program = y["text"]
        programs.append(program)
        for name in abstraction['function_name']:
            if name in program:
                print('🍊')
print(len(programs))



1148


In [62]:
for x in data["results_by_query"]:
    answer = x["parse_results"]
    for y in answer:
        program = y["text"]
        programs.append(program)
        for name in abstraction['function_name']:
            if name in program:
                print(program)
                print (y["valid"])
                if not y["valid"]:
                    print (y["error"])
                print ("---")

(lambda (regex_replace_any_letter $0 (regex_match 'e' $2) (regex_match 'i' $1)))
False
infer
---
(lambda (modify_word $0 '.' 'h' 'p'))
False
infer
---
(lambda (modify_word_based_on_consonant $0 (regex_flatten (regex_map (lambda (regex_if (regex_match 't' $0) 'l' $0)) (regex_split '.' $0))) (regex_flatten (regex_map (lambda (regex_if (regex_match 'e' $0) 'j' $0)) (regex_split '.' $0))))))
False
parse
---
(lambda (modify_word_based_on_consonant $0 (regex_flatten (regex_map (lambda (regex_if (regex_match 't' $0) 'l' $0)) (regex_split '.' $0))) (regex_flatten (regex_map (lambda (regex_if (regex_match 'e' $0) 'j' $0)) (regex_split '.' $0))))))
False
parse
---
(lambda (regex_if (regex_match '.' (regex_tail (regex_split empty_string $0))) (regex_flatten (regex_replace_any_letter (regex_flatten (regex_map (lambda (regex_if (regex_match 'm' $0) 'j' $0)) (regex_split empty_string $0))))) (regex_flatten (regex_map (lambda (regex_if (regex_match 'm' $0) 'j' $0)) (regex_split empty_string $0)))))
F